In [2]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np
# import agg_functions # only works once in Jupyter notebook after restart kernel. Using magic line instead
%run agg_functions.py

#Set seed
RSEED = 50

Agg_functions loaded!


In [2]:
# Read in data
app_train = pd.read_csv('../input/application_train.csv').replace({365243: np.nan})
app_test = pd.read_csv('../input/application_test.csv').replace({365243: np.nan})

train_ids = list(app_train['SK_ID_CURR'])
test_ids = list(app_test['SK_ID_CURR'])

# One Hot Encode
app_train, app_test = pd.get_dummies(app_train), pd.get_dummies(app_test)
app_test['TARGET'] = np.nan

# Align on the columns
app_train, app_test = app_train.align(app_test, axis = 1, join = 'inner')

original_features = list(app_train.columns)
original_features.remove('TARGET')
original_features.remove('SK_ID_CURR')

# Combine train/test into one dataset
app = app_train.append(app_test, sort=False)

#We join the training and testing data
#together to make sure that any operations we do to one set 
#is repeated for the other set. 
#The testing data can be extracted using 
#test = app[app["TARGET"].isnull()].copy()
#and the training data by
#train = app[app['TARGET'].notnull()].copy()
                                                                
# Extract the training and testing data
#app_new_train, app_new_test = app[app['TARGET'].notnull()], app[app['TARGET'].isnull()]


#app_train = pd.read_csv('../input/application_train.csv').replace({365243: np.nan})
#app_test = pd.read_csv('../input/application_test.csv').replace({365243: np.nan})
#bureau = pd.read_csv('../input/bureau.csv').replace({365243: np.nan})
#bureau_balance = pd.read_csv('../input/bureau_balance.csv').replace({365243: np.nan})
#cash = pd.read_csv('../input/POS_CASH_balance.csv').replace({365243: np.nan})
#credit = pd.read_csv('../input/credit_card_balance.csv').replace({365243: np.nan})
#previous = pd.read_csv('../input/previous_application.csv').replace({365243: np.nan})
#installments = pd.read_csv('../input/installments_payments.csv').replace({365243: np.nan})

In [3]:
bureau = pd.read_csv('../input/bureau.csv')
bureau_info = agg_child(bureau, 'SK_ID_CURR', 'BUREAU')

In [4]:
previous = pd.read_csv('../input/previous_application.csv')
previous_info = agg_child(previous, 'SK_ID_CURR', 'PREVIOUS')

In [5]:
bureau_balance = pd.read_csv('../input/bureau_balance.csv')
bureau_balance_info = agg_grandchild(bureau_balance, bureau, 'SK_ID_BUREAU', 'SK_ID_CURR', 'BB')
del bureau_balance

In [6]:
credit_card = pd.read_csv('../input/credit_card_balance.csv')
credit_card_info = agg_grandchild(credit_card, previous, 'SK_ID_PREV', 'SK_ID_CURR', 'CC')
del credit_card

In [7]:
cash = pd.read_csv('../input/POS_CASH_balance.csv')
cash_info = agg_grandchild(cash, previous, 'SK_ID_PREV', 'SK_ID_CURR', 'CASH')
del cash

In [8]:
installments = pd.read_csv('../input/installments_payments.csv')
installments_info = agg_grandchild(installments, previous, 'SK_ID_PREV', 'SK_ID_CURR', 'IN')
del installments

In [9]:
gc.enable()
del bureau, previous
gc.collect()

71

In [10]:
#Merge everything into train dataframe
app_train = app_train.set_index('SK_ID_CURR')
app_test = app_test.set_index('SK_ID_CURR')

In [11]:
app_train = app_train.merge(bureau_info, on = 'SK_ID_CURR', how = 'left')
app_test = app_test.merge(bureau_info, on = 'SK_ID_CURR', how = 'left')
del bureau_info

In [12]:
app_train = app_train.merge(previous_info, on = 'SK_ID_CURR', how = 'left')
app_test = app_test.merge(previous_info, on = 'SK_ID_CURR', how = 'left')
del previous_info

In [13]:
app_train = app_train.merge(bureau_balance_info, on = 'SK_ID_CURR', how = 'left')
app_test = app_test.merge(bureau_balance_info, on = 'SK_ID_CURR', how = 'left')
del bureau_balance_info

In [14]:
app_train = app_train.merge(credit_card_info, on = 'SK_ID_CURR', how = 'left')
app_test = app_test.merge(credit_card_info, on = 'SK_ID_CURR', how = 'left')
del credit_card_info

In [15]:
app_train = app_train.merge(cash_info, on = 'SK_ID_CURR', how = 'left')
app_test = app_test.merge(cash_info, on = 'SK_ID_CURR', how = 'left')
del cash_info

In [16]:
app_train = app_train.merge(installments_info, on = 'SK_ID_CURR', how = 'left')
app_test = app_test.merge(installments_info, on = 'SK_ID_CURR', how = 'left')
del installments_info

In [17]:
# Check for columns with duplicated values
#_, idx = np.unique(app_train, axis = 1, return_index = True)
#print('There are {} columns with all duplicated values.'.format(app.shape[1] - len(idx)))

#_, idx = np.unique(app_test, axis = 1, return_index = True)
#print('There are {} columns with all duplicated values.'.format(app.shape[1] - len(idx)))

In [18]:

app_train.reset_index(inplace=True)
app_test.reset_index(inplace=True)

app_train.to_csv('../input/app_train_inprogress.csv', index = False)
app_test.to_csv('../input/app_test_inprogress.csv', index = False)

In [3]:
app_train = pd.read_csv('../input/app_train_inprogress.csv')
app_test = pd.read_csv('../input/app_test_inprogress.csv')

In [7]:
print(app_train.shape)
print(app_test.shape)

(307511, 1417)
(48744, 1417)


In [ ]:
train_ids = list(app_train['SK_ID_CURR'])
test_ids = list(app_test['SK_ID_CURR'])

# One Hot Encode
app_train, app_test = pd.get_dummies(app_train), pd.get_dummies(app_test)

# Add nan-target variable to test in order to be able to align
app_test['TARGET'] = np.nan

# Align on the columns
app_train, app_test = app_train.align(app_test, axis = 1, join = 'inner')

original_features = list(app_train.columns)
original_features.remove('TARGET')
original_features.remove('SK_ID_CURR')

# Combine train/test into one dataset
app = app_train.append(app_test, sort=False)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(app_train, app_train["TARGET"]):
    print(train_index, test_index)
    strat_train_set = app_train.loc[train_index]
    strat_test_set = app_train.loc[test_index]
    

In [ ]:
import lightgbm as lgb

def cross_validate(train):
    """Compute cross validation ROC AUC of a gradient boosting model for a given training dataset"""
    
    # Extract the labels
    train_labels = np.array(train['TARGET'].astype(np.int32)).reshape((-1, ))
    train = train.drop(columns = ['TARGET', 'SK_ID_CURR'])

    # Create a  lgb training set
    train_set = lgb.Dataset(train, label = train_labels)

    # Find default hyperparameters
    model = lgb.LGBMClassifier()
    params = model.get_params()

    # Number of estimators will be selected through early stopping
    del params['n_estimators'], params['silent']

    # Early stoppping with 5 fold cross validation
    cv_results = lgb.cv(params, train_set, num_boost_round = 10000, metrics = 'auc', 
                        early_stopping_rounds = 100, seed = RSEED, nfold = 5)

    print('Cross Validation ROC AUC: {:.5f} with std: {:.5f}.'.format(cv_results['auc-mean'][-1],
                                                                               cv_results['auc-stdv'][-1]))

    print('Number of estimators trained: {}'.format(len(cv_results['auc-mean'])))
    
    return cv_results

In [ ]:
cv_results_baseline = cross_validate(strat_train_set)

In [ ]:
def run_verification(cv_results, train, test):
    
    # Extract the labels
    train_labels = np.array(train['TARGET'].astype(np.int32)).reshape((-1, ))
    train = train.drop(columns = ['TARGET', 'SK_ID_CURR'])
    test_ids = list(test['SK_ID_CURR'])
    test_target = test["TARGET"]
    test = test.drop(columns = ['TARGET', 'SK_ID_CURR']) 
    
    # Make model with optimal number of estimators and train on training data
    model = lgb.LGBMClassifier(n_estimators = len(cv_results['auc-mean']), random_state=RSEED)
    model.fit(train, train_labels)
    
    # Make predictions on the testing data
    prob = model.predict_proba(test)[:, 1]
    preds = prob.round()
    test_predictions = pd.DataFrame({'SK_ID_CURR': test_ids, 
                                'TARGET': test_target,
                                'PREDICTION': preds,
                                'PROBABILITY': prob})
    return test_predictions

In [ ]:
results = run_verification(cv_results_baseline, strat_train_set, strat_test_set)

In [ ]:
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc

# Compute Precision-Recall and plot curve
precision, recall, thresholds = precision_recall_curve(results["TARGET"], results["PROBABILITY"])
area = auc(recall, precision)
print("Area Under PR Curve(AP): %0.2f" % area)  #should be same as AP?

print('AP', average_precision_score(results["TARGET"], results["PROBABILITY"], average='weighted'))

In [ ]:
#Create submission file for Kaggle
submission_baseline = kaggle_submission(cv_results_baseline, app_train, app_test)
submission_baseline.to_csv('../input/submission_baseline.csv', index = False)

In [ ]:
import agg_functions as agg
agg.hello_world()